In [1]:
# Generic OpenAI-compatible endpoint using ChatOpenAI
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.messages import HumanMessage,SystemMessage
import os
default_model_name = os.environ["OPENAI_MODEL"]
pg_connection = os.environ["PGVECTOR_CONNECTION_STRING"]
llm = ChatOpenAI(model=default_model_name,temperature=0)

In [2]:
from langchain_ollama import OllamaEmbeddings 
from langchain_postgres.vectorstores import PGVector
 
 
# embed each chunk and insert it into the vector store
embeddings_model = OllamaEmbeddings(model="nomic-embed-text")
# Coba dengan nama default
try:
    db = PGVector.from_existing_index(
        embedding=embeddings_model,
        collection_name="langchain",  # Coba default
        connection=pg_connection,
    )
    print("Berhasil konek dengan collection 'langchain'")
except Exception as e1:
    print(f"Collection 'langchain' tidak ditemukan: {e1}")
    
    # Coba dengan nama lain yang mungkin
    try:
        db = PGVector.from_existing_index(
            embedding=embeddings_model,
            collection_name="nomic-embed-text",  # Nama model
            connection=pg_connection,
        )
        print("Berhasil konek dengan collection 'nomic-embed-text'")
    except Exception as e2:
        print(f"Collection 'nomic-embed-text' tidak ditemukan: {e2}")

Berhasil konek dengan collection 'langchain'


In [ ]:
db.similarity_search("Q3", k=4)

[Document(id='1677ab38-7604-4515-869a-ddf06cf5bae8', metadata={'source': '../../tesla.md'}, page_content='---\n\n PART III\n\nITEM 10\\. DIRECTORS, EXECUTIVE OFFICERS AND CORPORATE GOVERNANCE\n\nThe information required by this Item 10 of Form 10-K will be included in our 2023 Proxy Statement to be filed with the Securities and Exchange Commission in connection with the solicitation of proxies for our 2023 Annual Meeting of Stockholders and is incorporated herein by reference. The 2023 Proxy Statement will be filed with the Securities and Exchange Commission within 120 days after the end of the fiscal year to which this report relates. \n\nITEM 11\\. EXECUTIVE COMPENSATION\n\nThe information required by this Item 11 of Form 10-K will be included in our 2023 Proxy Statement and is incorporated herein by reference. \n\nITEM 12\\. SECURITY OWNERSHIP OF CERTAIN BENEFICIAL OWNERS AND MANAGEMENT AND RELATED STOCKHOLDER MATTERS\n\nThe information required by this Item 12 of Form 10-K will be 

In [ ]:
# create retriever
# retriever = db.as_retriever(search_kwargs={"k": 2})

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

retriever = db.as_retriever(search_kwargs={"k": 2})

prompt = ChatPromptTemplate.from_template("""Always answer in indonesian language.Answer the question based only on
    the following context:
{context}

Question: {question}
""")
chain = prompt | llm
rewrite_prompt = ChatPromptTemplate.from_template("""Provide a better search
    query for web search engine to answer the given question, end the queries
    with '**'.
    examples:
    Question: Minyak
    Answer: Jenis Minyak**
    Question: {x} Answer:""")

def parse_rewriter_output(message):
    return message.content.strip('"').strip("**")

rewriter = rewrite_prompt | llm | parse_rewriter_output

 
def qa_rrr(input):
    # rewrite the query
    new_query = rewriter.invoke(input)
    print(f"q:{new_query}")
    # fetch relevant documents
    docs = retriever.get_relevant_documents(new_query)
    # format prompt
    formatted = prompt.invoke({"context": docs, "question": input})
    # generate answer
    answer = llm.invoke(formatted)
    return answer
 
result=qa_rrr("Apa saja produk TESLA?")
print(result.content)

q:Daftar Produk Tesla lengkap


/home/cristminix/tmp-dir/ipykernel_1686218/3860650886.py:40: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  docs = retriever.get_relevant_documents(new_query)


Berdasarkan dokumen yang tersedia, produk Tesla meliputi:

1. **Kendaraan listrik** (mobil Tesla) — baik baru maupun bekas, yang dilengkapi dengan jaminan terbatas pabrikan dan rencana layanan diperpanjang untuk beberapa model di wilayah tertentu.
2. **Produk energi** — termasuk sistem penyimpanan energi (seperti baterai rumah), yang juga disertai jaminan terbatas serta layanan perbaikan.

Selain itu, Tesla juga menyediakan layanan pemasangan untuk sistem energi tersebut, lengkap dengan jaminan terhadap pekerjaan instalasi.


In [7]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Always answer in indonesian language.Answer the question based only on
    the following context:
{context}

Question: {question}
""")
retriever = db.as_retriever(search_kwargs={"k": 2})
perspectives_prompt = ChatPromptTemplate.from_template(
    """You are an AI language model assistant. Your task is to generate five different versions 
    of the given user question to retrieve relevant documents from a vector database.
    By generating multiple perspectives on the user question, your goal is to help the user 
    overcome some of the limitations of the distance-based similarity search. Provide these 
    alternative questions separated by newlines. Original question: {question}"""
)
def parse_queries_output(message):
    return message.content.split('\n')

query_gen = perspectives_prompt | llm | parse_queries_output

def get_unique_union(document_lists):
    # Flatten list of lists, and dedupe them
    deduped_docs = {
        doc.page_content: doc
        for sublist in document_lists for doc in sublist
    }
    # return a flat list of unique docs
    return list(deduped_docs.values())

retrieval_chain = query_gen | retriever.batch | get_unique_union

def multi_query_qa(input):
    # fetch relevant documents
    docs = retrieval_chain.invoke(input)
    # format prompt
    formatted = prompt.invoke({"context": docs, "question": input})
    # generate answer
    answer = llm.invoke(formatted)
    return answer

result=multi_query_qa("cabe hujau?")
print(result.content)

Cabai hijau adalah jenis cabai yang dipanen pada tahap awal kematangan, sebelum terjadi perubahan warna pada buah. Berdasarkan informasi dari dokumen, cabai hijau biasanya dipanen sekitar 70-80 hari setelah tanam. Pemanenan dilakukan pagi hari setelah embun kering atau sore hari untuk mengurangi stres pada tanaman. Untuk memastikan kualitas dan memperpanjang masa simpan, gunting atau pisau tajam digunakan untuk memotong tangkai buah, dengan meninggalkan sedikit tangkai pada buah.

Selain itu, penanganan pasca panen yang baik juga penting untuk menjaga kualitas cabai hijau, termasuk sortasi berdasarkan ukuran, warna, dan kondisi buah, pencucian untuk membersihkan kotoran, serta pengemasan yang memungkinkan sirkulasi udara.


In [6]:


#---------------------- START DISINI -------------------------------------
prompt_rag_fusion = ChatPromptTemplate.from_template("""Anda adalah asisten
    yang berguna untuk menghasilkan beberapa kata kunci pencarian berdasarkan
    satu pertanyaan masukan. \n
    Buatlah beberapa kata kunci pencarian yang berkaitan dengan: {question} \n
    Hasil (4 kueri):""")

def parse_queries_output(message):
    return message.content.split('\n')


query_gen = prompt_rag_fusion | llm | parse_queries_output


def reciprocal_rank_fusion(results: list[list], k=60):
    """Reciprocal Rank Fusion (RRF) diterapkan pada beberapa daftar dokumen 
       yang telah diperingkat, dengan parameter opsional k yang digunakan 
       dalam rumus RRF.
    """

    # Inisialisasi dictionary untuk menyimpan skor gabungan setiap dokumen
    # Dokumen akan diindeks berdasarkan isinya untuk memastikan keunikan
    fused_scores = {}
    documents = {}

    # Iterasi melalui setiap daftar dokumen yang telah diperingkat
    for docs in results:
        # Iterasi melalui setiap dokumen dalam daftar,
        # beserta peringkatnya (posisi dalam daftar)
        for rank, doc in enumerate(docs):
            # Gunakan isi dokumen sebagai kunci untuk memastikan keunikan
            doc_str = doc.page_content
            # Jika dokumen belum pernah ditemukan sebelumnya,
            # - inisialisasi skor ke 0
            # - simpan untuk digunakan nanti
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
                documents[doc_str] = doc
            # Perbarui skor dokumen menggunakan rumus RRF:
            # 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Urutkan dokumen berdasarkan skor gabungan secara menurun
    # untuk mendapatkan hasil peringkat ulang akhir
    reranked_doc_strs = sorted(
        fused_scores, key=lambda d : fused_scores[d], reverse=True
    )
    # Ambil dokumen yang sesuai untuk setiap doc_str
    return [
        documents[doc_str]
        for doc_str in reranked_doc_strs
    ]

retrieval_chain = query_gen | retriever.batch | reciprocal_rank_fusion

# Pemakaian
docs=retrieval_chain.invoke({"question":"cabe hujau?"})
print(docs)

[Document(id='5e2c2cc8-3ded-41ae-946a-08e00df69d4e', metadata={'source': '../../tesla.md'}, page_content='|         |  |                                                                                                                                                                                                                                                                                                                                                                                                                                                            |  |                           |  |           |  |         |  |                  |  |          |'), Document(id='8817e773-4adb-4b19-841d-dfeff0852c3a', metadata={'source': '../../test.txt'}, page_content='Pemanenan Cabai: Teknik dan Penanganan Pasca Panen\nIndikator dan Teknik Panen\nCabai dapat dipanen pada beberapa tahap kematangan tergantung tujuan pasar:\n\nCabai hijau - dipanen 70-80 hari setelah tanam, sebelum perubahan warna\

In [14]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser 

prompt_hyde = ChatPromptTemplate.from_template("""Tolong tulis sebuah paragraf untuk
   menjawab pertanyaan.\n Pertanyaan: {question} \n Paragraf:""")

generate_doc = (
    prompt_hyde | llm | StrOutputParser()
)
# generate_doc.invoke({"question":"Siapa elon?"})
retrieval_chain = generate_doc | retriever
# retrieval_chain.invoke({"question":"Siapa elon?"})

def qa(input):
  # ambil dokumen relevan dari rantai pengambilan hyde yang didefinisikan sebelumnya
  docs = retrieval_chain.invoke(input)
  # format prompt
  formatted = prompt.invoke({"context": docs, "question": input})
  # buat jawaban
  chain = llm|StrOutputParser()
  answer = chain.invoke(formatted)
  return answer

qa("Siapa tesla?")

'Berdasarkan dokumen yang diberikan, Tesla adalah sebuah perusahaan yang berfokus pada percepatan transisi dunia menuju energi berkelanjutan. Tujuan utama Tesla adalah mengatasi emisi karbon dengan menyediakan ekosistem energi dan transportasi yang lengkap melalui desain dan manufaktur kendaraan listrik serta produk penyimpanan energi.\n\nTesla memproduksi berbagai jenis kendaraan listrik, termasuk mobil penumpang seperti Model S, Model 3, Model X, dan Model Y, serta kendaraan komersial seperti Tesla Semi. Selain itu, Tesla juga merencanakan pengembangan kendaraan listrik spesial seperti Cybertruck dan Tesla Roadster baru.\n\nPerusahaan ini juga memproduksi produk penyimpanan energi berbasis baterai lithium-ion, yaitu Powerwall untuk rumah atau fasilitas komersial kecil, serta Megapack untuk pelanggan komersial, industri, utilitas, dan pembangkit energi. Produk-produk ini dirancang untuk mendukung sistem energi terbarukan dan meningkatkan efisiensi operasional pabrik-pabrik Tesla secar